In [1]:
import dotenv
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from langchain_openai import AzureOpenAIEmbeddings
dotenv.load_dotenv()
from langchain_community.vectorstores import Chroma
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_similarity,
    answer_correctness,
)
from datasets import Dataset
from langchain_openai import AzureChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from operator import itemgetter
from langchain.prompts import ChatPromptTemplate
import os
import pandas as pd
import numpy as np
import sys
sys.tracebacklimit = 0
from langchain_community.document_loaders import PyPDFLoader

c:\Users\sigitalapina\OneDrive - KPMG\Desktop\thesis-rag\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
OPENAI_API_VERSION = os.environ.get("OPENAI_API_VERSION")
AZURE_OPENAI_ENDPOINT = os.environ.get("AZURE_OPENAI_ENDPOINT")
OPENAI_MODEL = os.environ.get("OPENAI_MODEL")
OPENAI_DEPLOYMENT = os.environ.get("OPENAI_DEPLOYMENT")
EMBEDDING_MODEL = os.environ.get("EMBEDDING_MODEL")
EMBEDDING_DEPLOYMENT = os.environ.get("EMBEDDING_DEPLOYMENT")
OPENAI_MODEL_GPT4 = os.environ.get("OPENAI_MODEL_GPT4")
OPENAI_DEPLOYMENT_GPT4 = os.environ.get("OPENAI_DEPLOYMENT_GPT4")

In [3]:
embeddings_client = AzureOpenAIEmbeddings(
    azure_deployment=EMBEDDING_DEPLOYMENT,
    openai_api_version=OPENAI_API_VERSION)
llm = AzureChatOpenAI(model_name=OPENAI_MODEL, azure_deployment=OPENAI_DEPLOYMENT,temperature=0)
llm_gpt4 = AzureChatOpenAI(model_name=OPENAI_MODEL_GPT4, azure_deployment=OPENAI_DEPLOYMENT_GPT4,temperature=0)

In [4]:
def evaluation_llm(questions, answers, contexts, ground_truths):
    data = {
        "question": questions,
        "answer": answers,
        "contexts": contexts,
        "ground_truths": ground_truths
    }
    dataset = Dataset.from_dict(data)
    azure_configs = {
        "base_url": AZURE_OPENAI_ENDPOINT,
        "model_deployment": OPENAI_DEPLOYMENT,
        "model_name": OPENAI_MODEL,
        "embedding_deployment": EMBEDDING_DEPLOYMENT,
        "embedding_name": EMBEDDING_MODEL,  
    }

    azure_model = AzureChatOpenAI(
        openai_api_version=OPENAI_API_VERSION,
        azure_endpoint=azure_configs["base_url"],
        azure_deployment=azure_configs["model_deployment"],
        model=azure_configs["model_name"],
        validate_base_url=False,
    )

    azure_embeddings = AzureOpenAIEmbeddings(
        openai_api_version=OPENAI_API_VERSION,
        azure_endpoint=azure_configs["base_url"],
        azure_deployment=azure_configs["embedding_deployment"],
        model=azure_configs["embedding_name"],
    )
    result = evaluate(
        dataset = dataset, 
        metrics=[
            faithfulness,
            answer_relevancy,
            answer_similarity,
            answer_correctness,
        ], 
        llm=azure_model, 
        embeddings=azure_embeddings,
    )
    return result

In [5]:
def evaluation_rag(questions, answers, contexts, ground_truths):
    data = {
        "question": questions,
        "answer": answers,
        "contexts": contexts,
        "ground_truths": ground_truths
    }
    dataset = Dataset.from_dict(data)
    azure_configs = {
        "base_url": AZURE_OPENAI_ENDPOINT,
        "model_deployment": OPENAI_DEPLOYMENT,
        "model_name": OPENAI_MODEL,
        "embedding_deployment": EMBEDDING_DEPLOYMENT,
        "embedding_name": EMBEDDING_MODEL,  # most likely
    }

    azure_model = AzureChatOpenAI(
        openai_api_version=OPENAI_API_VERSION,
        azure_endpoint=azure_configs["base_url"],
        azure_deployment=azure_configs["model_deployment"],
        model=azure_configs["model_name"],
        validate_base_url=False,
    )

    azure_embeddings = AzureOpenAIEmbeddings(
        openai_api_version=OPENAI_API_VERSION,
        azure_endpoint=azure_configs["base_url"],
        azure_deployment=azure_configs["embedding_deployment"],
        model=azure_configs["embedding_name"],
    )
    result = evaluate(
        dataset = dataset, 
        metrics=[
            faithfulness,
            answer_relevancy,
            context_precision,
            context_recall,
            answer_similarity,
            answer_correctness,
        ], 
        llm=azure_model, 
        embeddings=azure_embeddings,
        raise_exceptions=False,
    )
    return result

In [6]:
columns = ["System", "Faithfulness", "Answer Relevancy", "Context Precision", "Context Recall", "Answer Similarity", "Answer Correctness"]
results_df = pd.DataFrame(columns=columns)

In [7]:
max_average = 0
def find_highest(average_score):
    global max_average
    if average_score > max_average:
        max_average = average_score
        print("This is the new best value!")

In [8]:
def dictionary(result):
    dict_result = dict(result)
    average_score = sum(dict_result.values()) / len(dict_result)
    print(f"The average score is: {average_score}")
    find_highest(average_score)
    return average_score

In [9]:
def evaluate_system(system_name, questions, answers, contexts, ground_truths):
    result = evaluation_rag(questions, answers, contexts, ground_truths)
    average = dictionary(result)
    # Create a dictionary to store the results
    system_results = {
        "System": system_name,
        "Faithfulness": result["faithfulness"],
        "Answer Relevancy": result["answer_relevancy"],
        "Context Precision": result["context_precision"],
        "Context Recall": result["context_recall"],
        "Answer Similarity": result["answer_similarity"],
        "Answer Correctness": result["answer_correctness"],
        "Average": average
    }
    df_system_results = pd.DataFrame([system_results])
    return df_system_results

In [10]:
def evaluate_LLM(system_name, questions, answers, contexts, ground_truths):
    result = evaluation_rag(questions, answers, contexts, ground_truths)
    average = dictionary(result)
    # Create a dictionary to store the results
    system_results = {
        "System": system_name,
        "Faithfulness": result["faithfulness"],
        "Answer Relevancy": result["answer_relevancy"],
        "Context Precision": np.nan,
        "Context Recall": np.nan,
        "Answer Similarity": result["answer_similarity"],
        "Answer Correctness": result["answer_correctness"],
        "Average": average
    }
    df_llm_results = pd.DataFrame([system_results])
    return df_llm_results

In [11]:
questions = [
    "Write a news report that starts as follows: The Preprint titled 'The Chronicles of RAG: The Retriever, the Chunk and the Generator' delves into cutting-edge strategies for implementing the Retrieval Augmented Generation (RAG) system tailored for the Brazilian Portuguese language. Written by a team of researchers from Brazil, the paper addresses the challenges and advancements in optimizing RAG systems for improved text generation in response to user queries.",
    "Write a short report that starts as follows: MultiHop-RAG is a benchmarking dataset designed for evaluating Retrieval-Augmented Generation (RAG) systems for multi-hop queries.",
    "Write a short report that starts as follows: In recent years, the integration of Large Language Models (LLMs) has significantly advanced generative AI services, notably with the Retrieval-Augmented Generation (RAG) model. This report initiates a comprehensive exploration, evaluating RAG's impact through the Retrieval-Augmented Generation Benchmark (RGB) and its application in the Dialog System Technology Challenge 9 (DSTC9). Additionally, we delve into innovative approaches like GENREAD, which enhances LLMs' capabilities for tasks such as form filling and multi-hop query resolution, aiming to provide actionable insights for leveraging these technologies effectively.",
    "Finish the following idea: The CRUD framework offers a structured approach to assessing the capabilities and limitations of Retrieval-Augmented Generation (RAG) systems in different text generation scenarios.",
    "Write a short report that starts as follows: HybridRAG is a novel approach that efficiently combines a cloud-based Large Language Model (LLM) with a smaller client-side language model through retrieval augmented memory to deliver real-time composition assistance. It allows the client model to generate effective responses by leveraging the LLM's capabilities and contextual information.",
    "Finish writing the following short report: MetaRAG is introduced as a framework that combines the retrieval-augmented generation process with metacognitive abilities, inspired by human cognitive processes. The integration of Metacognition in MetaRAG allows the model to monitor, evaluate, and plan its response strategies, enhancing introspective reasoning and self-awareness in the cognitive process. This approach enables the model to identify shortcomings in the reasoning process and aims to enhance the accuracy of answer derivation.",
    "Write a report that starts as follows: RAGTruth is a hallucination corpus specifically designed for analyzing word-level hallucinations in diverse tasks within the Retrieval-Augmented Generation (RAG) context for Language Models. This corpus encompasses nearly 18,000 naturally generated responses from various models using RAG. Each response has undergone meticulous manual annotation at both individual cases and word levels, evaluating the intensity of hallucinations.",
    "Write a short news report that starts as follows: The PaperQA system revolutionizes scientific research with its cutting-edge capabilities in answering complex scientific questions.",
    "Finish writing the following report: Various evaluation metrics play a crucial role in assessing the performance of Retrieval-Augmented Generation (RAG) systems across different text generation scenarios. Methods that can be used are reference-free, evaluation for retrievers, evaluation for embedding models, RagQuestEval Metrics, evaluation based on chunk size, evaluation of LLMs as well as end-to-end evaluation metrics.",
    "Finish writing the following report: Retrievers are a crucial component in the field of natural language processing, especially in the context of information retrieval and question answering systems. They play a key role in enabling models to access external knowledge sources efficiently and effectively. Retrievers are often utilized to retrieve and present relevant documents or passages from large corpora, such as Wikipedia, to aid in answering queries or generating responses. The integration of retrievers in language models, such as in the Framework Retrieval-Augmented Language Model (REALM), allows for the retrieval of knowledge from external sources during pre-training, fine-tuning, and inference stages."
]

ground_truths = [
    ["The Preprint titled 'The Chronicles of RAG: The Retriever, the Chunk and the Generator' delves into cutting-edge strategies for implementing the Retrieval Augmented Generation (RAG) system tailored for the Brazilian Portuguese language. Written by a team of researchers from Brazil, the paper addresses the challenges and advancements in optimizing RAG systems for improved text generation in response to user queries. In a meticulous review of RAG techniques, the researchers highlight the significance of effective integration of retrieval models, data representation learning, computational efficiency optimization, and the importance of quality text generation during RAG implementation. Through a series of experiments focused on the Harry Potter book dataset, different retrieval techniques were tested, including sparse and dense retrievers, sentence chunking strategies, and the impact of document positioning within prompts on overall content relevance. The results showcased a significant enhancement in the RAG system's quality, with a notable improvement in Mean Reciprocal Rank (MRR@10) and recall metrics when using hybrid search techniques that combine both sparse and dense retrieval methods. The hybrid approach demonstrated the potential to further enhance the accuracy and relevance of responses generated by RAG systems in Brazilian Portuguese-related tasks. Moreover, the study introduced a new metric called the relative maximum score, which allows for a direct comparison of the performance of different LLMs within the RAG system. The research aims to provide valuable insights and practical guidelines for developers looking to implement RAG in Brazilian Portuguese applications, with a focus on simplifying the pipeline for inference and experiments. Overall, the paper sheds light on the evolving landscape of RAG systems and offers key recommendations for optimizing performance and quality in Brazilian Portuguese text generation applications. This research opens new avenues for the application of advanced retrieval techniques in enhancing the capabilities of language models for diverse language tasks."],
    ["MultiHop-RAG is a benchmarking dataset designed for evaluating Retrieval-Augmented Generation (RAG) systems for multi-hop queries. This dataset includes a knowledge base, a collection of multi-hop queries, their ground-truth answers, and associated supporting evidence. Multi-hop queries are those that require retrieving and reasoning over multiple pieces of evidence from various sources to formulate a response. MultiHop-RAG addresses the limitations of existing RAG systems by focusing on complex multi-hop queries and providing a resource for developing effective RAG frameworks, ultimately enhancing the adoption of Large Language Models (LLMs) in practical applications. The dataset encompasses various query types, such as inference, comparison, temporal, and null queries, each testing the retrieval and reasoning capabilities of the LLMs in handling complex information processing tasks. Additionally, MultiHop-RAG facilitates benchmarking of RAG systems by offering a diverse set of queries and evidential support challenging the retrieval and reasoning abilities of various state-of-the-art LLMs. The dataset represents a valuable resource for the NLP community to advance RAG technologies and harness the potential of generative AI in real-world applications."],
    ["In recent years, the integration of Large Language Models (LLMs) has significantly advanced generative AI services, notably with the Retrieval-Augmented Generation (RAG) model. This report initiates a comprehensive exploration, evaluating RAG's impact through the Retrieval-Augmented Generation Benchmark (RGB) and its application in the Dialog System Technology Challenge 9 (DSTC9). Additionally, we delve into innovative approaches like GENREAD, which enhances LLMs' capabilities for tasks such as form filling and multi-hop query resolution, aiming to provide actionable insights for leveraging these technologies effectively. Generative AI Services: The study proposes a method for implementing generative AI services using Large Language Models (LLMs). Addressing Information Scarcity: The research addresses the challenge of information scarcity by utilizing LLM capabilities and proposes remedies for inadequate data. Fine-tuning and Document Integration: Strategies like fine-tuning techniques and direct document integration are explored to mitigate data insufficiency. Retrieval-Augmented Generation (RAG): Introduces the RAG model designed to enhance information storage, retrieval processes, and content generation. Information Storage and Retrieval Methodology: The study outlines key phases of the information storage and retrieval methodology supported by the RAG model. Evaluation of RAG Impact on LLMs: Analyzes the impact of Retrieval-Augmented Generation on different LLMs, focusing on abilities like noise robustness, negative rejection, information integration, and counterfactual robustness. Retrieval-Augmented Generation Benchmark (RGB): A corpus for evaluating RAG in English and Chinese, divided into testbeds for various abilities required for RAG. Dialog System Technology Challenge (DSTC 9): Discusses work on the DSTC 9 track focused on task-oriented conversational modeling with unstructured knowledge access. Automated Form Filling: Proposes adapting LLMs for automated form completion tasks by creating a knowledge base, augmenting LLMs with retrieval-augmented generation, and using prompt engineering techniques. Generate-then-Read (GENREAD): Presents a method for solving knowledge-intensive tasks by generating contextual documents using LLMs before reading them for answers. Hybrid Retrieval-Augmented Generation (HybridRAG): Introduces a framework combining cloud-based LLMs with client-side models through retrieval-augmented memory for real-time task efficiency. MetaRAG: Integrates retrieval-augmented generation with metacognition to enhance model reasoning abilities through self-reflection and critical evaluation. MultiHop-RAG Dataset: Develops a dataset focusing on multi-hop queries to address inadequacies in existing RAG systems, aiming to facilitate the development of effective RAG systems for LLMs."],
    ["The CRUD framework offers a structured approach to assessing the capabilities and limitations of Retrieval-Augmented Generation (RAG) systems in different text generation scenarios. By categorizing tasks into Create, Read, Update, and Delete types, CRUD provides a comprehensive evaluation method for RAG systems. This framework allows for targeted analysis of how external knowledge sources enhance text generation in various contexts, enabling researchers and developers to optimize RAG systems more effectively."],
    ["HybridRAG is a novel approach that efficiently combines a cloud-based Large Language Model (LLM) with a smaller client-side language model through retrieval augmented memory to deliver real-time composition assistance. It allows the client model to generate effective responses by leveraging the LLM's capabilities and contextual information. The framework comprises four main components: 1. Augmentation coordinator (client): Monitors the writing context, requests augmented memory from the cloud, and integrates it into the client model 2. Memory-augmented client model (client): Utilizes the augmented memory to suggest the next words, enhancing its overall performance. 3. Retriever model (cloud): Searches the retrieval corpus to find relevant data based on the user input. 4. LLM-based memory generator (cloud): Constructs concise key takeaways from the retrieved data to optimize memory usefulness. The HybridRAG framework offers multiple benefits: 1. Enhanced utility: Enables the client model to provide better suggestions by utilizing cloud-based resources. 2. Low latency: Ensures low latency by asynchronously updating memory, allowing the client model to make predictions swiftly without waiting for responses from the cloud. 3. Reduced client-to-cloud communication: Minimizes communication between the client and cloud, saving costs by only requesting augmented memory when needed. 4. Efficient memory integration: Integrates cloud-generated memory into the client model to enhance performance and reduce data transfer volume. By leveraging cloud-generated memory, HybridRAG efficiently enhances the utility of small language models on client devices through retrieval augmentation, operating asynchronously. This innovative framework bridges the gap between latency challenges and model performance, making it a valuable solution for real-time tasks like composition assistance."],
    ["MetaRAG is introduced as a framework that combines the retrieval-augmented generation process with metacognitive abilities, inspired by human cognitive processes. The integration of Metacognition in MetaRAG allows the model to monitor, evaluate, and plan its response strategies, enhancing introspective reasoning and self-awareness in the cognitive process. This approach enables the model to identify shortcomings in the reasoning process and aims to enhance the accuracy of answer derivation. The overall MetaRAG framework includes two spaces: the cognition space and the metacognition space. The cognition space functions as a Question Answering (QA) system, while the metacognition space serves as both an evaluator and critic, introspecting the reasoning process. The metacognition space primarily encompasses three main phases: Monitoring, Evaluating, and Planning. The monitoring phase assesses the quality of the current response to determine the need for metacognitive evaluation. The evaluating phase identifies reasons why the current answer may not meet the requirements, utilizing metacognitive knowledge to analyze flaws in the response. In summary, MetaRAG is a framework that integrates metacognition into retrieval-augmented generation processes. By leveraging these metacognitive capabilities, the model can enhance its reasoning abilities, identify errors in its responses, and optimize its cognitive processes for more accurate answer generation."],
    ["RAGTruth is a hallucination corpus specifically designed for analyzing word-level hallucinations in diverse tasks within the Retrieval-Augmented Generation (RAG) context for Language Models. This corpus encompasses nearly 18,000 naturally generated responses from various models using RAG. Each response has undergone meticulous manual annotation at both individual cases and word levels, evaluating the intensity of hallucinations. RAGTruth serves as a benchmark dataset to measure the extent of hallucination within Large Language Models (LLMs) and assess existing strategies for hallucination detection. It provides a foundation for developing effective hallucination prevention methods under RAG scenarios, aiming to enhance the trustworthiness of LLM applications. With a focus on the effectiveness of hallucination detection methodologies, RAGTruth enables thorough comparisons across different LLMs and tasks. Additionally, the report emphasizes the significance of high-quality datasets like RAGTruth in fine-tuning relatively smaller LLMs to achieve competitive performance in hallucination detection, underscoring the potential for advancing detection methods using specialized datasets. With detailed annotations and diverse responses across tasks, RAGTruth facilitates a comprehensive understanding of hallucination phenomena in LLMs, contributing to the ongoing research in this critical area."],
    ["The PaperQA system revolutionizes scientific research with its cutting-edge capabilities in answering complex scientific questions. Developed by a team of researchers, PaperQA utilizes advanced tools and technologies to retrieve relevant information from online databases like Arxiv and Pubmed. By synthesizing data from research papers, PaperQA provides accurate answers comparable to human experts. Its innovative approach includes decomposing tasks, utilizing map summarization, and leveraging advanced language models to enhance retrieval and reasoning processes. PaperQA's cost-effectiveness, accuracy, and reduced error rates make it a valuable asset for scientific exploration. This breakthrough system promises to expedite research processes and reduce costs in the natural sciences, paving the way for faster innovation and discovery."],
    ["Various evaluation metrics play a crucial role in assessing the performance of Retrieval-Augmented Generation (RAG) systems across different text generation scenarios.  Methods that can be used are reference-free, evaluation for retrievers, evaluation for embedding models, RagQuestEval Metrics, evaluation based on chunk size, evaluation of LLMs as well as end-to-end evaluation metrics. Reference-Free Evaluation Methods: The utilization of metrics like TruLens-Eval, RAGAS, and ARES is discussed for evaluating RAG systems' performance concerning context relevance, answer faithfulness, and answer relevance. Evaluation of Retrievers: Experimental results incorporate metrics such as BLEU, ROUGE-L, bertScore, and RAGQuestEval to evaluate different retrievers' performance in tasks like summarization and question-answering. Evaluation of Embedding Models: Experimental results evaluate different embedding models based on metrics such as BLEU, ROUGE-L, bertScore, and RAGQuestEval for tasks like text continuation and summarization. RAGQuestEval Metric: The RAGQuestEval metric measures the recall and precision of key information in the generated text compared to the ground truth references. Evaluation based on Chunk Size: Evaluation of different chunk sizes utilizes metrics like BLEU, ROUGE-L, bertScore, and RAGQuestEval for text continuation and summarization tasks. Evaluation of Large Language Models: Experimental results showcase the evaluation of different large language models using metrics such as BLEU, ROUGE-L, bertScore, and RAGQuestEval for text continuation and summarization tasks. End-to-End Evaluation Method: An end-to-end evaluation method is adopted to assess RAG systems' ability to retrieve relevant documents and generate sensible responses, comparing the similarity between model output and reference answers. Moreover, the report introduces the Retrieval-Augmented Generation Benchmark (RGB) to evaluate Large Language Models' (LLMs) abilities in retrieval-augmented generation tasks. The benchmark requires RAG systems to exhibit Noise Robustness, Negative Rejection, Information Integration, and Counterfactual Robustness for accurate responses. Data construction involves using actual news articles to simulate real-world scenarios for evaluating LLMs by judging their retrieval-augmented responses to questions. Evaluation metrics used in the RGB include Accuracy for noise robustness and information integration, Rejection Rate for negative rejection, and Error Detection Rate for counterfactual robustness. The paper also discusses the impact of noise on LLMs' performance in RAG tasks, highlighting common errors such as long-distance information issues, evidence uncertainty, and concept confusion."],
    ["Retrievers are a crucial component in the field of natural language processing, especially in the context of information retrieval and question answering systems. They play a key role in enabling models to access external knowledge sources efficiently and effectively. Retrievers are often utilized to retrieve and present relevant documents or passages from large corpora, such as Wikipedia, to aid in answering queries or generating responses. The integration of retrievers in language models, such as in the Framework Retrieval-Augmented Language Model (REALM), allows for the retrieval of knowledge from external sources during pre-training, fine-tuning, and inference stages. This retrieval process involves selecting and attending over documents from a corpus to inform the predictions or responses generated by the model. Retrievers in systems like REALM are trained using unsupervised techniques, such as masked language modeling as the learning signal, and backpropagation through a retrieval step that considers millions of documents. The retriever is designed to reward informative retrievals that improve model predictions and penalize uninformative retrievals, thereby encouraging the model to retrieve relevant knowledge. One of the main challenges in incorporating retrievers in neural networks is the computational complexity involved, especially when considering a large corpus of documents. Strategies such as caching and asynchronous updates, as well as approaches like Maximum Inner Product Search (MIPS), are employed to address these challenges and optimize the retrieval process efficiently. Furthermore, retrievers are instrumental in tasks like open-domain question answering, where models need to access a vast amount of external knowledge to provide accurate responses. By enabling models to retrieve and incorporate relevant information from a knowledge corpus, retrievers enhance the performance and accuracy of the overall system, as demonstrated by the significant improvements achieved in benchmarks like NaturalQuestions-Open, WebQuestions, and CuratedTrec. In conclusion, retrievers are fundamental components in language models that facilitate the retrieval of external knowledge for various NLP tasks. Their effective integration and training are essential for enhancing model performance, enabling better understanding, and interpretation of textual data across a wide range of applications."]
]

### General answer by LLMs

In [12]:
template = """{question}"""
prompt = ChatPromptTemplate.from_template(template)

In [13]:
llm_chain =(
    { "question": itemgetter("question")}
    | RunnablePassthrough()
    | {"response": prompt | llm}
)
llm_chain_gpt4 =(
    { "question": itemgetter("question")}
    | RunnablePassthrough()
    | {"response": prompt | llm_gpt4}
)

In [14]:
answers_llm = []
contexts_llm = [[""],[""],[""],[""],[""],[""],[""],[""],[""],[""]]

In [15]:
for query in questions:
    response = llm_chain.invoke({"question": query})
    answers_llm.append(response["response"].content)

In [16]:
llm_results = evaluate_LLM("GPT-3.5", questions, answers_llm, contexts_llm, ground_truths)
results_df = pd.concat([results_df, llm_results], ignore_index=True)
print(llm_results)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:  15%|█▌        | 9/60 [00:04<00:21,  2.41it/s]Runner in Executor raised an exception
TypeError: expected string or buffer
Runner in Executor raised an exception
TypeError: expected string or buffer
Evaluating: 100%|██████████| 60/60 [00:13<00:00,  4.53it/s]


The average score is: 0.831677071290973
This is the new best value!
    System  Faithfulness  Answer Relevancy  Context Precision  Context Recall  \
0  GPT-3.5      0.817609          0.795673                NaN             NaN   

   Answer Similarity  Answer Correctness   Average  
0           0.984847             0.64856  0.831677  


C:\Users\sigitalapina\AppData\Local\Temp\ipykernel_5388\800954320.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, llm_results], ignore_index=True)


In [17]:
answers_llm_gpt4 = []
for query in questions:
    response = llm_chain_gpt4.invoke({"question": query})
    answers_llm_gpt4.append(response["response"].content)
llm_results_gpt4 = evaluate_LLM("GPT-4",questions, answers_llm_gpt4, contexts_llm, ground_truths)
results_df = pd.concat([results_df, llm_results_gpt4], ignore_index=True)
print(llm_results_gpt4)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:  15%|█▌        | 9/60 [00:04<00:21,  2.42it/s]Runner in Executor raised an exception
TypeError: expected string or buffer
Runner in Executor raised an exception
TypeError: expected string or buffer
Evaluating: 100%|██████████| 60/60 [00:10<00:00,  5.76it/s]


The average score is: 0.8381448027141722
This is the new best value!
  System  Faithfulness  Answer Relevancy  Context Precision  Context Recall  \
0  GPT-4      0.837413          0.873383                NaN             NaN   

   Answer Similarity  Answer Correctness   Average  
0           0.911056            0.667323  0.838145  


# RAG

### Naive RAG

In [18]:
template = """User input {question}. 
Context {context}."""
prompt = ChatPromptTemplate.from_template(template)

In [19]:
def retrieval_chain(prompt, retriever, llm):
    retrieval_augmented_qa_chain = (
        {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
        | RunnablePassthrough.assign(context=itemgetter("context"))
        | {"response": prompt | llm, "context": itemgetter("context")}
    )
    return retrieval_augmented_qa_chain

In [20]:
loader = DirectoryLoader('../papers', glob="./*.pdf", loader_cls=PyPDFLoader)
documents = loader.load()

In [21]:
# THE FIRST TIME RUN THIS, AFTER RUN THE NEXT CELL
# text_splitter = CharacterTextSplitter()
# chunks = text_splitter.split_documents(documents)
# db_naive = Chroma.from_documents(chunks, embeddings_client, persist_directory = "../papers/vectordb-edit/naive")
# db_naive.persist()
# retriever_naive = db_naive.as_retriever()

In [22]:
db_naive = Chroma(persist_directory = "../papers/vectordb-edit/naive", embedding_function=embeddings_client)
retriever_naive = db_naive.as_retriever()

In [23]:
answers_naive = []
contexts_naive = []
for query in questions:
    try:  
        response = retrieval_chain(prompt, retriever_naive, llm).invoke({"question": query})
        # Access the response content
        answers_naive.append(response["response"].content)
        # Access the context content
        context_content = [context.page_content for context in response["context"]]
        contexts_naive.append(context_content)  
    except Exception as e:  
        print(f"Warning: {e}" + "on the following question: " + query)  
        answers_naive.append("No answer")
        context_full = retriever_naive.get_relevant_documents(query)
        context_content = [context.page_content for context in context_full]
        contexts_naive.append(context_content)

In [24]:
result_naive_rag = evaluate_system("Naive", questions, answers_naive, contexts_naive, ground_truths)
results_df = pd.concat([results_df, result_naive_rag], ignore_index=True)
print(result_naive_rag)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 60/60 [00:11<00:00,  5.06it/s]


The average score is: 0.8819782610333614
This is the new best value!
  System  Faithfulness  Answer Relevancy  Context Precision  Context Recall  \
0  Naive      0.987654          0.860312                1.0           0.925   

   Answer Similarity  Answer Correctness   Average  
0           0.975971            0.542932  0.881978  


### recursive splitter

In [25]:
# THE FIRST TIME RUN THIS, AFTER RUN THE NEXT CELL
# text_splitter = text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder()
# chunks_r = text_splitter.split_documents(documents)
# db_basic = Chroma.from_documents(chunks_r, embeddings_client, persist_directory = "../papers/vectordb-edit/recursive_basic")
# db_basic.persist()
# retriever_basic = db_basic.as_retriever()

In [26]:
db_basic = Chroma(persist_directory = "../papers/vectordb-edit/recursive_basic", embedding_function=embeddings_client)
retriever_basic = db_basic.as_retriever()

In [27]:
answers_recursive = []
contexts_recursive = []
for query in questions:
    try:  
        response = retrieval_chain(prompt, retriever_basic, llm).invoke({"question": query})
        # Access the response content
        answers_recursive.append(response["response"].content)
        # Access the context content
        context_content = [context.page_content for context in response["context"]]
        contexts_recursive.append(context_content)  
    except Exception as e:  
        print(f"Warning: {e}" + "on the following question: " + query)  
        answers_recursive.append("No answer")
        context_full = retriever_basic.get_relevant_documents(query)
        context_content = [context.page_content for context in context_full]
        contexts_recursive.append(context_content)

In [28]:
result_recursive = evaluate_system("Recursive", questions, answers_naive, contexts_naive, ground_truths)
results_df = pd.concat([results_df, result_recursive], ignore_index=True)
result_recursive

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 60/60 [00:29<00:00,  2.04it/s]


The average score is: 0.8842059449363263
This is the new best value!


,System,Faithfulness,Answer Relevancy,Context Precision,Context Recall,Answer Similarity,Answer Correctness,Average
0,Recursive,0.987654,0.85812,1.0,0.95,0.975971,0.533491,0.884206


### chunk sizes

In [29]:
def run_and_evaluate(name, retriever, prompt, llm, results_df):
    answers = []
    contexts_extra = []

    for query in questions:
        try:  
            response = retrieval_chain(prompt, retriever, llm).invoke({"question": query})
            # Access the response content
            answers.append(response["response"].content)
            # Access the context content
            context_content = [context.page_content for context in response["context"]]
            contexts_extra.append(context_content)  
        except Exception as e:  
            print(f"Warning: {e}" + "on the following question: " + query)  
            answers.append("No answer")
            context_full = retriever.get_relevant_documents(query)
            context_content = [context.page_content for context in context_full]
            contexts_extra.append(context_content)

    result = evaluate_system(name, questions, answers, contexts_extra, ground_truths)
    results_df = pd.concat([results_df, result], ignore_index=True)
    return result, results_df

In [30]:
chunk_sizes = [500, 1000, 2000, 3000]
overlap_percentages = [0, 5, 10, 15, 20]

for chunk_size in chunk_sizes:
    for overlap_percentage in overlap_percentages:
        # Calculate overlap based on percentage
        chunk_overlap = int(chunk_size * overlap_percentage / 100)
        
        # Create text splitter
        # text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        
        # # Split documents
        # chunks = text_splitter.split_documents(documents)
        
        # Print number of chunks
        print(f"Number of chunks for chunk size {chunk_size}, overlap {overlap_percentage}%")
        
        # Create Chroma database
        # db = Chroma.from_documents(chunks, embeddings_client, persist_directory=f"../papers/vectordb-edit/chunking_{chunk_size}_{overlap_percentage}")
        db = Chroma(persist_directory = f"../papers/vectordb-edit/chunking_{chunk_size}_{overlap_percentage}", embedding_function=embeddings_client)
        db.persist()
        
        # Create retriever
        retriever = db.as_retriever()
        
        # Run and evaluate
        result,results_df = run_and_evaluate(f"Chunk {chunk_size}, overlap {overlap_percentage}%", retriever, prompt, llm, results_df)
        print(f"CHUNK SIZE {chunk_size}, {overlap_percentage}% overlap:")
        print(result)

Number of chunks for chunk size 500, overlap 0%


passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]Task exception was never retrieved
future: <Task finished name='Task-293' coro=<AsyncClient.aclose() done, defined at c:\Users\sigitalapina\OneDrive - KPMG\Desktop\thesis-rag\.venv\Lib\site-packages\httpx\_client.py:2011> exception=RuntimeError('Event loop is closed')>
RuntimeError: Event loop is closed
Task exception was never retrieved
future: <Task finished name='Task-294' coro=<AsyncClient.aclose() done, defined at c:\Users\sigitalapina\OneDrive - KPMG\Desktop\thesis-rag\.venv\Lib\site-packages\httpx\_client.py:2011> exception=RuntimeError('Event loop is closed')>
RuntimeError: Event loop is closed
Task exception was never retrieved
future: <Task finished name='Task-295' coro=<AsyncClient.aclose() done, defined

The average score is: 0.8892122732658302
This is the new best value!
CHUNK SIZE 500, 0% overlap:
                  System  Faithfulness  Answer Relevancy  Context Precision  \
0  Chunk 500, overlap 0%          0.93          0.877987           0.983333   

   Context Recall  Answer Similarity  Answer Correctness   Average  
0             1.0           0.978099            0.565855  0.889212  
Number of chunks for chunk size 500, overlap 5%


passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 60/60 [00:27<00:00,  2.21it/s]


The average score is: 0.8860495705054383
CHUNK SIZE 500, 5% overlap:
                  System  Faithfulness  Answer Relevancy  Context Precision  \
0  Chunk 500, overlap 5%      0.936667          0.882256                1.0   

   Context Recall  Answer Similarity  Answer Correctness  Average  
0        0.966667           0.981681            0.549027  0.88605  
Number of chunks for chunk size 500, overlap 10%


...

The average score is: 0.8809038231015536
CHUNK SIZE 3000, 10% overlap:
                    System  Faithfulness  Answer Relevancy  Context Precision  \
0  Chunk 3000, overlap 10%           1.0          0.851984                1.0   

   Context Recall  Answer Similarity  Answer Correctness   Average  
0           0.925           0.973843            0.534596  0.880904  
Number of chunks for chunk size 3000, overlap 15%


passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]Task exception was never retrieved
future: <Task finished name='Task-1488' coro=<AsyncClient.aclose() done, defined at c:\Users\sigitalapina\OneDrive - KPMG\Desktop\thesis-rag\.venv\Lib\site-packages\httpx\_client.py:2011> exception=RuntimeError('Event loop is closed')>
RuntimeError: Event loop is closed
Task exception was never retrieved
future: <Task finished name='Task-1489' coro=<AsyncClient.aclose() done, defined at c:\Users\sigitalapina\OneDrive - KPMG\Desktop\thesis-rag\.venv\Lib\site-packages\httpx\_client.py:2011> exception=RuntimeError('Event loop is closed')>
RuntimeError: Event loop is closed
Task exception was never retrieved
future: <Task finished name='Task-1490' coro=<AsyncClient.aclose() done, defi

The average score is: 0.8785177117409474
CHUNK SIZE 3000, 15% overlap:
                    System  Faithfulness  Answer Relevancy  Context Precision  \
0  Chunk 3000, overlap 15%      0.987654          0.850571           0.972222   

   Context Recall  Answer Similarity  Answer Correctness   Average  
0            0.95            0.97589            0.534769  0.878518  
Number of chunks for chunk size 3000, overlap 20%


passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 60/60 [00:10<00:00,  5.63it/s]


The average score is: 0.8850949783891008
CHUNK SIZE 3000, 20% overlap:
                    System  Faithfulness  Answer Relevancy  Context Precision  \
0  Chunk 3000, overlap 20%           1.0          0.852384           0.991667   

   Context Recall  Answer Similarity  Answer Correctness   Average  
0            0.95           0.975239            0.541281  0.885095  


chunk = 1000

In [31]:
# # THE FIRST TIME RUN THIS, AFTER RUN THE NEXT CELL
# text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size = 1000, chunk_overlap = 100)
# chunks_1000 = text_splitter.split_documents(documents)
# print(len(chunks_1000))
# db_1000 = Chroma.from_documents(chunks_1000, embeddings_client, persist_directory = "../papers/vectordb-edit/recursive_1000")
# db_1000.persist()

In [32]:
# db_1000 = Chroma(persist_directory = "../papers/vectordb-edit/recursive_1000", embedding_function=embeddings_client)

In [33]:
# retriever_1000 = db_1000.as_retriever()
# result_1000 = run_and_evaluate(retriever_1000, prompt, llm)
# print("CHUNK SIZE 1000")
# print(result_1000)

In [34]:
# dict_result_1000 = dictionary(result_1000)

chunk = 500

In [35]:
# # THE FIRST TIME RUN THIS, AFTER RUN THE NEXT CELL
# text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size = 500, chunk_overlap = 50)
# chunks_500 = text_splitter.split_documents(documents)
# print(len(chunks_500))
# db_500 = Chroma.from_documents(chunks_500, embeddings_client, persist_directory = "../papers/vectordb-edit/recursive_500")
# db_500.persist()

In [36]:
# db_500 = Chroma(persist_directory = "../papers/vectordb-edit/recursive_500", embedding_function=embeddings_client)

In [37]:
# retriever_500 = db_500.as_retriever()
# result_500 = run_and_evaluate(retriever_500, prompt, llm)
# print("CHUNK SIZE 500")
# print(result_500)

In [38]:
# dict_result_500 = dictionary(result_500)

chunk = 2000

In [39]:
# text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size = 2000, chunk_overlap = 200)
# chunks_2000 = text_splitter.split_documents(documents)
# print(len(chunks_2000))
# db_2000 = Chroma.from_documents(chunks_2000, embeddings_client, persist_directory = "../papers/vectordb-edit/recursive_2000")


In [40]:
# db_2000 = Chroma(persist_directory = "../papers/vectordb-edit/recursive_2000", embedding_function=embeddings_client)

In [41]:
# retriever_2000 = db_2000.as_retriever()
# result_2000 = run_and_evaluate(retriever_2000, prompt, llm)
# print("CHUNK SIZE 2000")
# print(result_2000)

In [42]:
# dict_result_2000 = dictionary(result_2000)

chunk = 3000

In [43]:
# text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size = 3000, chunk_overlap = 300)
# chunks_3000 = text_splitter.split_documents(documents)
# print(len(chunks_3000))
# db_3000 = Chroma.from_documents(chunks_3000, embeddings_client, persist_directory = "../papers/vectordb-edit/recursive_3000")

In [44]:
# db_3000 = Chroma(persist_directory = "../papers/vectordb-edit/recursive_3000", embedding_function=embeddings_client)

In [45]:
# retriever_3000 = db_3000.as_retriever()
# result_3000 = run_and_evaluate(retriever_3000, prompt, llm)
# print("CHUNK SIZE 3000")
# print(result_3000)

In [46]:
# dict_result_3000 = dictionary(result_3000)

In [49]:
highest = results_df.nlargest(1, "Average")
highest

,System,Faithfulness,Answer Relevancy,Context Precision,Context Recall,Answer Similarity,Answer Correctness,Average
7,"Chunk 500, overlap 15%",0.987654,0.872242,0.983333,1.0,0.977634,0.61061,0.905246


In [48]:
results_df.to_csv(f"../papers/results/results_reporting.csv", index=False)

### now time to look for different top-k

Note: We continue with the size chunk of 1000 as it had the highest average score

In [50]:
db_k = Chroma(persist_directory = "../papers/vectordb-edit/chunking_500_15", embedding_function=embeddings_client)

In [51]:
k_values = [2, 3, 5, 6, 7]

# Iterate over different k values
for k in k_values:
    # Create retriever with k value
    retriever = db_k.as_retriever(search_kwargs={"k": k})
    
    # Run and evaluate
    result,results_df = run_and_evaluate(f"Chunk 500, overlap 15%, K={k}", retriever, prompt, llm, results_df)
    print(f"Results for K={k}:")
    print(result)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 60/60 [00:10<00:00,  5.66it/s]


The average score is: 0.8984579430671776
Results for K=2:
                        System  Faithfulness  Answer Relevancy  \
0  Chunk 500, overlap 15%, K=2      0.945833          0.848406   

   Context Precision  Context Recall  Answer Similarity  Answer Correctness  \
0                1.0        0.972115           0.970987            0.653406   

    Average  
0  0.898458  


passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:  28%|██▊       | 17/60 [00:04<00:10,  3.94it/s]Runner in Executor raised an exception
TypeError: expected string or buffer
Runner in Executor raised an exception
TypeError: expected string or buffer
Evaluating: 100%|██████████| 60/60 [00:10<00:00,  5.59it/s]


The average score is: 0.8884631335065118
Results for K=3:
                        System  Faithfulness  Answer Relevancy  \
0  Chunk 500, overlap 15%, K=3      0.882586          0.701695   

   Context Precision  Context Recall  Answer Similarity  Answer Correctness  \
0           0.939808        0.925281            0.99146            0.889948   

    Average  
0  0.888463  


passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 60/60 [00:10<00:00,  5.55it/s]


The average score is: 0.905995933811031
This is the new best value!
Results for K=5:
                        System  Faithfulness  Answer Relevancy  \
0  Chunk 500, overlap 15%, K=5      0.965608          0.873658   

   Context Precision  Context Recall  Answer Similarity  Answer Correctness  \
0           0.967083             1.0           0.980351            0.649275   

    Average  
0  0.905996  


passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 60/60 [00:09<00:00,  6.54it/s]


The average score is: 0.9052579905677297
Results for K=6:
                        System  Faithfulness  Answer Relevancy  \
0  Chunk 500, overlap 15%, K=6      0.983333          0.865678   

   Context Precision  Context Recall  Answer Similarity  Answer Correctness  \
0           0.952222             1.0           0.979103            0.651211   

    Average  
0  0.905258  


passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 60/60 [00:16<00:00,  3.62it/s]


The average score is: 0.89344124229841
Results for K=7:
                        System  Faithfulness  Answer Relevancy  \
0  Chunk 500, overlap 15%, K=7      0.983333          0.874736   

   Context Precision  Context Recall  Answer Similarity  Answer Correctness  \
0            0.94996             1.0           0.979863            0.572755   

    Average  
0  0.893441  


In [ ]:
# retriever_3 = db_1000.as_retriever(search_kwargs={"k": 3})
# result_3 = run_and_evaluate(retriever_3, prompt, llm)
# print("CHUNK SIZE 1000, K=3")
# print(result_3)

In [ ]:
# dict_result_3 = dictionary(result_3)

In [ ]:
# retriever_5 = db_1000.as_retriever(search_kwargs={"k": 5})
# result_5 = run_and_evaluate(retriever_5, prompt, llm)
# print("CHUNK SIZE 1000, K=5")
# print(result_5)

In [ ]:
# dict_result_5 = dictionary(result_5)

In [ ]:
# retriever_6= db_1000.as_retriever(search_kwargs={"k": 6})
# result_6 = run_and_evaluate(retriever_6, prompt, llm)
# print("CHUNK SIZE 1000, K=5")
# print(result_6)

In [ ]:
# dict_result_6 = dictionary(result_6)

In [ ]:
# retriever_7= db_1000.as_retriever(search_kwargs={"k": 7})
# result_7 = run_and_evaluate(retriever_7, prompt, llm)
# print("CHUNK SIZE 1000, K=7")
# print(result_7)
# dict_result_7 = dictionary(result_7)

In [53]:
highest = results_df.nlargest(1, "Average")
highest

,System,Faithfulness,Answer Relevancy,Context Precision,Context Recall,Answer Similarity,Answer Correctness,Average
26,"Chunk 500, overlap 15%, K=5",0.965608,0.873658,0.967083,1.0,0.980351,0.649275,0.905996


### look for different retrievers


#### parent document retriever

In [62]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore

parent_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=1000, chunk_overlap = 200)
child_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=200, chunk_overlap = 0)

vectorstore = Chroma(collection_name="split_parents",persist_directory = "../papers/vectordb-edit/parent-report", embedding_function=embeddings_client)
store = InMemoryStore()
parent_document_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)
parent_document_retriever.add_documents(documents)
result_parent, results_df = run_and_evaluate(f"Parent Retriever 1000-200", parent_document_retriever, prompt, llm, results_df)
print(result_parent)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:  20%|██        | 12/60 [00:05<00:22,  2.14it/s]Runner in Executor raised an exception
TypeError: expected string or buffer
Evaluating: 100%|██████████| 60/60 [00:10<00:00,  5.56it/s]


The average score is: 0.8949698260662626
                      System  Faithfulness  Answer Relevancy  \
0  Parent Retriever 1000-200      0.848293          0.913773   

   Context Precision  Context Recall  Answer Similarity  Answer Correctness  \
0           0.957171             1.0           0.978647            0.671934   

   Average  
0  0.89497  


In [66]:
parent_splitter_small = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=500, chunk_overlap = 50)
child_splitter_small = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=100, chunk_overlap = 0)

vectorstore = Chroma(collection_name="split_parents_small",persist_directory = "../papers/vectordb-edit/parent_small-summary", embedding_function=embeddings_client)
store = InMemoryStore()
parent_document_retriever_small = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter_small,
    parent_splitter=parent_splitter_small,
)
parent_document_retriever_small.add_documents(documents)
result_parent_small, results_df = run_and_evaluate(f"Parent Retriever 500-100", parent_document_retriever_small, prompt, llm, results_df)
print(result_parent_small)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:  10%|█         | 6/60 [00:02<00:19,  2.77it/s]Runner in Executor raised an exception
TypeError: expected string or buffer
Evaluating: 100%|██████████| 60/60 [00:10<00:00,  5.59it/s]


The average score is: 0.8848851364272386
                     System  Faithfulness  Answer Relevancy  \
0  Parent Retriever 500-100       0.83553          0.901562   

   Context Precision  Context Recall  Answer Similarity  Answer Correctness  \
0           0.912543        0.958333           0.983373             0.71797   

    Average  
0  0.884885  


In [67]:
parent_splitter_large = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=1500, chunk_overlap = 150)
child_splitter_large = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=200, chunk_overlap = 0)

vectorstore = Chroma(collection_name="split_parents_large",persist_directory = "../papers/vectordb-edit/parent_large-summary", embedding_function=embeddings_client)
store = InMemoryStore()
parent_document_retriever_large = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter_large,
    parent_splitter=parent_splitter_large,
)
parent_document_retriever_large.add_documents(documents)
result_parent_large , results_df = run_and_evaluate(f"Parent Retriever 1500-200", parent_document_retriever_small, prompt, llm, results_df)
print(result_parent_large)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 60/60 [00:11<00:00,  5.44it/s]


The average score is: 0.9019985310619424
                      System  Faithfulness  Answer Relevancy  \
0  Parent Retriever 1500-200          0.96          0.884597   

   Context Precision  Context Recall  Answer Similarity  Answer Correctness  \
0           0.947222             1.0           0.978609            0.641562   

    Average  
0  0.901999  


In [63]:
highest = results_df.nlargest(1, "Average")
highest

,System,Faithfulness,Answer Relevancy,Context Precision,Context Recall,Answer Similarity,Answer Correctness,Average
26,"Chunk 500, overlap 15%, K=5",0.965608,0.873658,0.967083,1.0,0.980351,0.649275,0.905996


#### Maximum marginal relevance retrieval

In [54]:
retriever_mmr = db_k.as_retriever(search_type="mmr",search_kwargs={"k": 5})
result_mmr, results_df = run_and_evaluate(f"MMR", retriever_mmr, prompt, llm, results_df)
print("Marginal relevance")
print(result_mmr)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:  22%|██▏       | 13/60 [00:04<00:16,  2.84it/s]Runner in Executor raised an exception
TypeError: expected string or buffer
Evaluating: 100%|██████████| 60/60 [00:11<00:00,  5.12it/s]


The average score is: 0.8638864433945551
Marginal relevance
  System  Faithfulness  Answer Relevancy  Context Precision  Context Recall  \
0    MMR      0.802754          0.933615           0.854385        0.891667   

   Answer Similarity  Answer Correctness   Average  
0           0.984872            0.716026  0.863886  


#### BM25

In [55]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size = 500, chunk_overlap = 75)
chunks_2000 = text_splitter.split_documents(documents)

In [56]:
from langchain.retrievers import BM25Retriever
retriever_bm25 = BM25Retriever.from_documents(chunks_2000, search_kwargs={"k": 5})
result_bm25, results_df = run_and_evaluate(f"BM25", retriever_bm25, prompt, llm, results_df)
print("BM25")
print(result_bm25)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:  18%|█▊        | 11/60 [00:02<00:09,  5.13it/s]Runner in Executor raised an exception
TypeError: expected string or buffer
Evaluating: 100%|██████████| 60/60 [00:12<00:00,  4.97it/s]


The average score is: 0.890429581684527
BM25
  System  Faithfulness  Answer Relevancy  Context Precision  Context Recall  \
0   BM25      0.937017          0.890066           0.922367           0.945   

   Answer Similarity  Answer Correctness  Average  
0           0.983907             0.66422  0.89043  


#### Ensambler - Hybrid

In [57]:
ret = db_k.as_retriever(search_kwargs={"k": 5})

In [58]:
from langchain.retrievers import EnsembleRetriever
ensemble_retriever_1 = EnsembleRetriever(retrievers=[retriever_bm25, ret], weights=[0.75, 0.25])
result_ensemble1, results_df = run_and_evaluate(f"Ensambler 1", ensemble_retriever_1, prompt, llm, results_df)
print("Ensambler")
print(result_ensemble1)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:  20%|██        | 12/60 [00:04<00:21,  2.27it/s]Runner in Executor raised an exception
TypeError: expected string or buffer
Evaluating: 100%|██████████| 60/60 [00:12<00:00,  4.93it/s]


The average score is: 0.8902747453103342
Ensambler
        System  Faithfulness  Answer Relevancy  Context Precision  \
0  Ensambler 1           1.0          0.887128           0.918554   

   Context Recall  Answer Similarity  Answer Correctness   Average  
0             1.0           0.972405            0.563561  0.890275  


In [59]:
ensemble_retriever_2 = EnsembleRetriever(retrievers=[retriever_bm25, ret], weights=[0.5, 0.5])
result_ensemble2, results_df = run_and_evaluate(f"Ensambler 2", ensemble_retriever_2, prompt, llm, results_df)
print("Ensambler")
print(result_ensemble2)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 60/60 [00:15<00:00,  3.81it/s]


The average score is: 0.8970552267707715
Ensambler
        System  Faithfulness  Answer Relevancy  Context Precision  \
0  Ensambler 2          0.98          0.855641           0.939841   

   Context Recall  Answer Similarity  Answer Correctness   Average  
0             1.0           0.975301            0.631548  0.897055  


In [60]:
ensemble_retriever_3 = EnsembleRetriever(retrievers=[retriever_bm25, ret], weights=[0.25,0.75])
result_ensemble3, results_df = run_and_evaluate(f"Ensambler 3", ensemble_retriever_3, prompt, llm, results_df)
print("Ensambler")
print(result_ensemble3)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 60/60 [00:17<00:00,  3.43it/s]


The average score is: 0.8772832957287428
Ensambler
        System  Faithfulness  Answer Relevancy  Context Precision  \
0  Ensambler 3      0.957143          0.851376           0.942063   

   Context Recall  Answer Similarity  Answer Correctness   Average  
0             1.0           0.972925            0.540192  0.877283  


#### Multi-stage - reranker

In [ ]:
os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")

In [ ]:
retriever_context = retriever_mmr
compressor = CohereRerank(top_n=6)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever_context
)

result_compression = run_and_evaluate(compression_retriever, prompt, llm)
print("Reranker")
print(result_compression)
avg_result_compression = dictionary(result_compression)

#### creating context by remaking the query

In [68]:
highest = results_df.nlargest(1, "Average")
highest

,System,Faithfulness,Answer Relevancy,Context Precision,Context Recall,Answer Similarity,Answer Correctness,Average
26,"Chunk 500, overlap 15%, K=5",0.965608,0.873658,0.967083,1.0,0.980351,0.649275,0.905996


In [70]:
template_context = "Generate a search query to fetch the relevant documents using the user's {question}. Craft a query that specifically targets the keywords in the question. In the answer provide only the query."
prompt_context = ChatPromptTemplate.from_template(template_context)

In [71]:
answers_final = []
contexts_final = []
# retriever_context_q = EnsembleRetriever(retrievers=[retriever_bm25, retriever_3], weights=[0.5, 0.5])
llm_for_context =(
    { "question": itemgetter("question")}
    | RunnablePassthrough()
    | {"response": prompt_context | llm}
)
for query in questions:
    response_check = llm_for_context.invoke({"question": query})
    search_query = response_check["response"].content
    retrieval_augmented_qa_chain = (
        {"context": itemgetter("context"), "question": itemgetter("question")}
        | RunnablePassthrough.assign(context=itemgetter("context"))
        | {"response": prompt | llm, "context": itemgetter("context")}
)
    docs = ret.get_relevant_documents(search_query)
    formatted_docs = []
    for doc in docs:
        resulting_doc = doc.page_content
        formatted_docs.append(resulting_doc)
    try:  
            response = retrieval_augmented_qa_chain.invoke({"context": formatted_docs, "question": query})
            # Access the response content
            answers_final.append(response["response"].content)
            contexts_final.append(formatted_docs)  
    except Exception as e:  
            print(f"Warning: {e}" + "on the following question: " + query)  
            answers_final.append("No answer")
            contexts_final.append(formatted_docs)


result_search_query = evaluation_rag(questions, answers_final, contexts_final, ground_truths)
result_search_query

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:  22%|██▏       | 13/60 [00:05<00:21,  2.23it/s]Runner in Executor raised an exception
TypeError: expected string or buffer
Evaluating: 100%|██████████| 60/60 [00:12<00:00,  4.78it/s]


{'faithfulness': 0.7289, 'answer_relevancy': 0.9634, 'context_precision': 0.8745, 'context_recall': 1.0000, 'answer_similarity': 0.9953, 'answer_correctness': 0.9217}

In [72]:
average = dictionary(result_search_query)
    # Create a dictionary to store the results
system_results = {
        "System": "Search query",
        "Faithfulness": result_search_query["faithfulness"],
        "Answer Relevancy": result_search_query["answer_relevancy"],
        "Context Precision": result_search_query["context_precision"],
        "Context Recall": result_search_query["context_recall"],
        "Answer Similarity": result_search_query["answer_similarity"],
        "Answer Correctness": result_search_query["answer_correctness"],
        "Average": average
    }
df_result_search_query = pd.DataFrame([system_results])

The average score is: 0.9139344985923543
This is the new best value!


In [73]:
results_df = pd.concat([results_df, df_result_search_query], ignore_index=True)
results_df

,System,Faithfulness,Answer Relevancy,Context Precision,Context Recall,Answer Similarity,Answer Correctness,Average
0,GPT-3.5,0.817609,0.795673,NaN,NaN,0.984847,0.648560,0.831677
1,GPT-4,0.837413,0.873383,NaN,NaN,0.911056,0.667323,0.838145
2,Naive,0.987654,0.860312,1.000000,0.925000,0.975971,0.542932,0.881978
3,Recursive,0.987654,0.858120,1.000000,0.950000,0.975971,0.533491,0.884206
4,"Chunk 500, overlap 0%",0.930000,0.877987,0.983333,1.000000,0.978099,0.565855,0.889212
5,"Chunk 500, overlap 5%",0.936667,0.882256,1.000000,0.966667,0.981681,0.549027,0.886050
6,"Chunk 500, overlap 10%",0.940476,0.874959,0.991667,1.000000,0.977794,0.581521,0.894403
7,"Chunk 500, overlap 15%",0.987654,0.872242,0.983333,1.000000,0.977634,0.610610,0.905246
8,"Chunk 500, overlap 20%",0.935556,0.892518,0.991667,0.950000,0.981038,0.615212,0.894332
9,"Chunk 1000, overlap 0%",0.641996,0.931064,0.866918,0.991667,0.939922,0.875984,0.874592


### change model to GPT-4

In [74]:
top_3_highest = results_df.nlargest(3, "Average")
top_3_highest

,System,Faithfulness,Answer Relevancy,Context Precision,Context Recall,Answer Similarity,Answer Correctness,Average
37,Search query,0.728868,0.963367,0.874454,1.0,0.995261,0.921657,0.913934
26,"Chunk 500, overlap 15%, K=5",0.965608,0.873658,0.967083,1.0,0.980351,0.649275,0.905996
27,"Chunk 500, overlap 15%, K=6",0.983333,0.865678,0.952222,1.0,0.979103,0.651211,0.905258


In [75]:
result_k5_500_15_gpt4, results_df = run_and_evaluate(f"Chunk size 500, overlap 15%, K=3, GPT-4", ret, prompt, llm_gpt4, results_df)
result_k5_500_15_gpt4

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 60/60 [00:14<00:00,  4.16it/s]


The average score is: 0.9060101034823765


,System,Faithfulness,Answer Relevancy,Context Precision,Context Recall,Answer Similarity,Answer Correctness,Average
0,"Chunk size 500, overlap 15%, K=3, GPT-4",0.98,0.882142,0.97,1.0,0.969684,0.634235,0.90601


In [78]:
replacement_map = {
    'Chunk size 500, overlap 15%, K=3, GPT-4': 'Chunk size 500, overlap 15%, K=5, GPT-4	'
}
results_df['System'] = results_df['System'].replace(replacement_map)

In [76]:
ret_6 = db_k.as_retriever(search_kwargs={"k": 6})

In [77]:
result_k6_500_15_gpt4, results_df = run_and_evaluate(f"Chunk size 2000, overlap 10%, K=6, GPT-4", ret_6, prompt, llm_gpt4, results_df)
result_k6_500_15_gpt4

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:  17%|█▋        | 10/60 [00:02<00:09,  5.13it/s]Runner in Executor raised an exception
TypeError: expected string or buffer
Evaluating: 100%|██████████| 60/60 [00:12<00:00,  4.69it/s]


The average score is: 0.8987401472001592


,System,Faithfulness,Answer Relevancy,Context Precision,Context Recall,Answer Similarity,Answer Correctness,Average
0,"Chunk size 2000, overlap 10%, K=6, GPT-4",0.81067,0.930504,0.945079,0.986667,0.974901,0.74462,0.89874


In [79]:
answers_final = []
contexts_final = []
# retriever_context_q = EnsembleRetriever(retrievers=[retriever_bm25, retriever_3], weights=[0.5, 0.5])
llm_for_context =(
    { "question": itemgetter("question")}
    | RunnablePassthrough()
    | {"response": prompt_context | llm_gpt4}
)
for query in questions:
    response_check = llm_for_context.invoke({"question": query})
    search_query = response_check["response"].content
    retrieval_augmented_qa_chain = (
        {"context": itemgetter("context"), "question": itemgetter("question")}
        | RunnablePassthrough.assign(context=itemgetter("context"))
        | {"response": prompt | llm_gpt4, "context": itemgetter("context")}
)
    docs = ret.get_relevant_documents(search_query)
    formatted_docs = []
    for doc in docs:
        resulting_doc = doc.page_content
        formatted_docs.append(resulting_doc)
    try:  
            response = retrieval_augmented_qa_chain.invoke({"context": formatted_docs, "question": query})
            # Access the response content
            answers_final.append(response["response"].content)
            contexts_final.append(formatted_docs)  
    except Exception as e:  
            print(f"Warning: {e}" + "on the following question: " + query)  
            answers_final.append("No answer")
            contexts_final.append(formatted_docs)


result_search_query_gpt4 = evaluation_rag(questions, answers_final, contexts_final, ground_truths)
result_search_query_gpt4

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:  22%|██▏       | 13/60 [00:04<00:17,  2.64it/s]Runner in Executor raised an exception
TypeError: expected string or buffer
Runner in Executor raised an exception
TypeError: expected string or buffer
Evaluating:  32%|███▏      | 19/60 [00:05<00:09,  4.40it/s]Runner in Executor raised an exception
TypeError: expected string or buffer
Evaluating: 100%|██████████| 60/60 [00:11<00:00,  5.36it/s]


{'faithfulness': 0.9485, 'answer_relevancy': 0.8255, 'context_precision': 0.8207, 'context_recall': 0.8986, 'answer_similarity': 0.9381, 'answer_correctness': 0.8848}

In [82]:
average = dictionary(result_search_query_gpt4)
    # Create a dictionary to store the results
system_results = {
        "System": "Search query GPT4",
        "Faithfulness": result_search_query_gpt4["faithfulness"],
        "Answer Relevancy": result_search_query_gpt4["answer_relevancy"],
        "Context Precision": result_search_query_gpt4["context_precision"],
        "Context Recall": result_search_query_gpt4["context_recall"],
        "Answer Similarity": result_search_query_gpt4["answer_similarity"],
        "Answer Correctness": result_search_query_gpt4["answer_correctness"],
        "Average": average
    }
df_result_search_query = pd.DataFrame([system_results])
results_df = pd.concat([results_df, df_result_search_query], ignore_index=True)
results_df

The average score is: 0.8860074455725125


,System,Faithfulness,Answer Relevancy,Context Precision,Context Recall,Answer Similarity,Answer Correctness,Average
0,GPT-3.5,0.817609,0.795673,NaN,NaN,0.984847,0.648560,0.831677
1,GPT-4,0.837413,0.873383,NaN,NaN,0.911056,0.667323,0.838145
2,Naive,0.987654,0.860312,1.000000,0.925000,0.975971,0.542932,0.881978
3,Recursive,0.987654,0.858120,1.000000,0.950000,0.975971,0.533491,0.884206
4,"Chunk 500, overlap 0%",0.930000,0.877987,0.983333,1.000000,0.978099,0.565855,0.889212
5,"Chunk 500, overlap 5%",0.936667,0.882256,1.000000,0.966667,0.981681,0.549027,0.886050
6,"Chunk 500, overlap 10%",0.940476,0.874959,0.991667,1.000000,0.977794,0.581521,0.894403
7,"Chunk 500, overlap 15%",0.987654,0.872242,0.983333,1.000000,0.977634,0.610610,0.905246
8,"Chunk 500, overlap 20%",0.935556,0.892518,0.991667,0.950000,0.981038,0.615212,0.894332
9,"Chunk 1000, overlap 0%",0.641996,0.931064,0.866918,0.991667,0.939922,0.875984,0.874592


In [83]:
results_df.to_csv(f"../papers/results/results_reporting.csv")